In [29]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self,func):
        self.creator = func
        

In [30]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

In [31]:
class Square(Function):
    def forward(self, input):
        x = input **2
        return x
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
    

In [32]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [33]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

In [34]:
a= A(x)
b =B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [35]:

y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [36]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [37]:
A = a.creator
x = A.input
x.grad  = A.backward(a.grad)

In [38]:
print(x.grad)

3.297442541400256


In [47]:
### backward 메서드 추가 

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # 함수 가져오고
        if f is not None:
            x= f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [52]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
